In [237]:
import pandas as pd
import numpy as np

In [2]:
movies=pd.read_csv("tmdb_5000_movies.csv")
credits=pd.read_csv("tmdb_5000_credits.csv")

NameError: name 'pd' is not defined

In [239]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [240]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [241]:
movies=movies.merge(credits,on='title')

In [242]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [243]:
# now we would extract useful columns only
# genres, id, keywords, title, overview, cast, crew
movies=movies[['genres','id','keywords','title','overview','cast','crew']]

In [244]:
# preprocessing
# 1) removing null values
movies.isnull().sum()

genres      0
id          0
keywords    0
title       0
overview    3
cast        0
crew        0
dtype: int64

In [245]:
movies.dropna(inplace=True)

In [246]:
# 2) removing duplicates 
movies.duplicated().sum()

0

In [247]:
movies.iloc[0].genres   # genres of first movie

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [248]:
obj=movies.iloc[0].genres

In [249]:
# # we just want genre of movie like action, adventure etc
# def convert(obj):
#     l=[]
#     for i in obj:
#         l.append(i['name'])
#     return l

In [250]:
# l=convert(obj)   it will give error

In [251]:
# we cant pass name as index in line l.append(i['name'])
# to solve this we use ast.literal_eval()
import ast
def convert(obj):        # here obj is in form of string
    l=[]
    for i in ast.literal_eval(obj):      # here he obj converted into list
        l.append(i['name'])
    return l
 

In [252]:
movies['genres']=movies['genres'].apply(convert)     # we converted the genre column in the format we needed

In [253]:
# doing same with keyword column 
movies['keywords']=movies['keywords'].apply(convert)

In [254]:
def convert2(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter!=3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [255]:
# now the cast column
# the cast column has dictionaries ...where each dictionary contain info about one actor
# we would extract first three dictionaries i.e., first three actors
movies['cast']=movies['cast'].apply(convert2)

In [256]:
# now crew column...in this column we want value where job= director
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break                  # if you got job =director , you can break form that dictionary and move ahead to next dictionary
    return L

movies['crew']=movies['crew'].apply(fetch_director)

In [257]:
movies.head(1)

,genres,id,keywords,title,overview,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [258]:
movies['overview'][0]    # column overview is in form of string ..we would convert it into list

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [259]:
movies['overview']=movies['overview'].apply(lambda x:x.split())


In [260]:
movies.head(2)

,genres,id,keywords,title,overview,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]


In [261]:
# now we would remove spaces b/w words (Sam Worthington to SamWorthington ), so that there is no confusion b/w two names starting with Sam
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [262]:
# now we will create a new column named tag , that will concatenate all the above columns

In [263]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [264]:
# now we can remove some columns as we have added them in tag
new_df=movies[['id','title','tags']]

In [310]:
new_df.head(2)

,id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."


In [266]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

/var/folders/pb/n_vw224s15x84bv1x600p8w00000gn/T/ipykernel_42413/487797088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [267]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

/var/folders/pb/n_vw224s15x84bv1x600p8w00000gn/T/ipykernel_42413/4224080999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


In [268]:
# now there would be words that have similar meaning like activity and activities  , actor and actress , action and actions 
# we would keep only one out of them
# this is called stemming
import nltk
!pip install nltk

In [269]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [270]:
def stem(text):
    y=[]

    for i in text.split():
        y.append(ps.stem(i))     # ps.stem() is a pre-defined function that returns stem word

    return " ".join(y)

In [271]:
new_df['tags']=new_df['tags'].apply(stem)

/var/folders/pb/n_vw224s15x84bv1x600p8w00000gn/T/ipykernel_42413/3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [272]:
new_df['tags'][0]

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

In [273]:
# now our task is to find similarities between tags
# we would remove the stop words like - from, are , and etc
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')    # top 5000 most frequent words except stop_words would be counted

In [274]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [275]:
vectors[0]   # first movie....we would see many 0's as may be frequency of that word in that movie is zero

array([0, 0, 0, ..., 0, 0, 0])

In [276]:
cv.get_feature_names_out()  

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [279]:
# we have 5000 words ... so 5000 dimensional plot would be used ...
# the movies close to each other on plot are similar 
# we would calculate cosine distance 
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vectors)    # calculate dsitance of each movie with every other movie

In [280]:
similarity[0]   # calculate distnace of first movie with every other movie 
# like distance of first movie with itself would be 1 , hence diagnol would be 1 (every movie has similarity=1 with itself)

array([1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
       0.        ])

In [ ]:
# now we sort the similaroes in descending order
#  sorted(similarity[0],reverse=True)  if we do this , we would loss the links between movies
# to solve this,we would use...,
# list(enumerate(similarity[0]))    # it shows 0th movie dist with 0th,1st,2nd...movies ,where first column is movie number and 2nd is distance

In [ ]:
# sorted(list(enumerate(similarity[0])),reverse=True)    # here we dont loss links between movies
# but still we would get sorted order on basis of movie number like last movie would be on first...we want to sort on basis of distance

In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])

In [ ]:
# first five most similar movies
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

In [ ]:
# now we would make a function that will recommend 5 similar movies in response to a movie

# new_df[new_df['title']=='Batman'].index[0]

# Filtering the DataFrame:
# new_df[new_df['title']=='Batman']: filters new_df to create a new DataFrame that only contains rows where title column = 'Batman'.

# Accessing the Index:
# .index: return index (row labels) of the filtered df. result will be a Pandas Index obj containing indices of rows that matched the condtn.

# Getting the First Index:
# [0]: access first element of the Index object. index[0] gets the first index of the rows that have the title 'Batman'.

In [308]:
def recommend(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [309]:
recommend('Avatar')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.


In [312]:
# to send this file ro website
import pickle
pickle.dump(new_df,open('movies.pkl','wb'))